In [4]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, matthews_corrcoef, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [21]:
protein = "MYO7A" # GJB2
filename = f'C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\gene_specific_df\\{protein}_with_position.csv'
data = pd.read_csv(f"{filename}")
dot_index = filename.index('_')
gene = filename[:dot_index]
if gene == "combined" or gene == "combined_with_source":
    gene = "7 genes"
if gene == "features":
    gene = "190 genes"

data

,gene,variant,pathogenicity,uniprot_id,stability_WT,stability_MUT,blosum,hydrophobicity_WT,hydrophobicity_MUT,volume_WT,...,oda_MUT,sasa_WT,sasa_MUT,RSA_WT,RSA_MUT,oda_delta,sasa_delta,pssm,entropy,position
0,MYO7A,M1H,benign,Q13402,651.486,651.793,-2,1.9,-3.2,162.9,...,0.05,234.13,215.31,1.153350,0.996806,3.05,-18.82,5.096,2.240,1
1,MYO7A,M1L,benign,Q13402,651.486,651.544,2,1.9,3.8,162.9,...,-3.94,234.13,194.46,1.153350,1.018115,-0.94,-39.67,5.096,2.240,1
2,MYO7A,M1P,benign,Q13402,651.486,651.847,-2,1.9,-1.6,162.9,...,-5.26,234.13,177.16,1.153350,1.150390,-2.26,-56.97,5.096,2.240,1
3,MYO7A,M1T,benign,Q13402,651.486,651.816,-1,1.9,-0.7,162.9,...,-0.89,234.13,173.00,1.153350,1.061350,2.11,-61.13,5.096,2.240,1
4,MYO7A,M1W,benign,Q13402,651.486,651.273,-1,1.9,-0.9,162.9,...,-14.56,234.13,260.26,1.153350,0.985833,-11.56,26.13,5.096,2.240,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1397,MYO7A,P1724A,benign,NaN,647.276,648.577,-1,-1.6,1.8,112.7,...,-0.85,96.07,88.06,0.623831,0.727769,0.60,-8.01,4.731,1.311,1724
1398,MYO7A,I1157V,benign,NaN,643.906,645.169,3,4.5,4.2,166.7,...,-0.07,0.00,0.44,0.000000,0.002667,-0.07,0.44,4.479,0.295,1157
1399,MYO7A,R1602Q,benign,NaN,649.842,649.217,1,-4.5,-3.5,173.4,...,1.05,61.85,44.18,0.233396,0.206449,-1.90,-17.67,4.137,0.805,1602
1400,MYO7A,L16S,benign,NaN,650.713,650.329,-2,3.8,-0.8,166.7,...,-0.19,128.13,71.41,0.670838,0.499371,1.18,-56.72,-1.468,1.714,16


In [22]:
data = pd.get_dummies(data, columns=["secondary_structure"])

mapping = {"benign": 0, "pathogenic": 1}
data["pathogenicity"] = data["pathogenicity"].map(mapping)

data = data.drop(
    labels=["uniprot_id", "stability_WT", "stability_MUT", "hydrophobicity_WT", "hydrophobicity_MUT", "volume_WT",
            "volume_MUT", "sequence_length", "oda_MUT", "oda_WT", "sasa_WT", "sasa_MUT", "RSA_MUT", "variant", "gene",
            "protein_contain_transmembrane", "is_residue_transmembranal", "aa_WT", "aa_MUT"], axis=1, inplace=False)

data

,pathogenicity,blosum,plddt_residue,stability_delta,hydrophobicity_delta,volume_delta,RSA_WT,oda_delta,sasa_delta,pssm,entropy,position,secondary_structure_Beta strand,secondary_structure_Helix,secondary_structure_Loop,secondary_structure_Turn
0,0,-2,44.57,0.307,-5.1,-9.7,1.153350,3.05,-18.82,5.096,2.240,1,False,False,True,False
1,0,2,44.57,0.058,1.9,3.8,1.153350,-0.94,-39.67,5.096,2.240,1,False,False,True,False
2,0,-2,44.57,0.361,-3.5,-50.2,1.153350,-2.26,-56.97,5.096,2.240,1,False,False,True,False
3,0,-1,44.57,0.330,-2.6,-46.8,1.153350,2.11,-61.13,5.096,2.240,1,False,False,True,False
4,0,-1,44.57,-0.213,-2.8,64.9,1.153350,-11.56,26.13,5.096,2.240,1,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1397,0,-1,70.33,1.301,3.4,-24.1,0.623831,0.60,-8.01,4.731,1.311,1724,False,False,True,False
1398,0,3,88.86,1.263,-0.3,-26.7,0.000000,-0.07,0.44,4.479,0.295,1157,False,False,True,False
1399,0,1,88.15,-0.625,1.0,-29.6,0.233396,-1.90,-17.67,4.137,0.805,1602,False,False,True,False
1400,0,-2,62.85,-0.384,-4.6,-77.7,0.670838,1.18,-56.72,-1.468,1.714,16,False,False,True,False


In [23]:
# structural_features = ["oda_delta", "stability_delta", "secondary_structure_Beta strand", "secondary_structure_Helix", "secondary_structure_Loop", "sasa_delta", "RSA_WT"]
# # Drop structural features
# data = data.drop(labels=["stability_delta"], axis=1, inplace=False)

In [ ]:

# Create list of all positions.
positions = data["position"].unique()
probabilities = []
predictions = []
y_tests = []
tps = []
fps = []
fns = []
tns = []
errors = []
mistakes = 0

counter = 0

for pos in positions:
    counter += 1
    print(f"Position: {pos} ({counter}/{len(positions)})")
    # Create train and test sets.
    train = data[data["position"] != pos]
    test = data[data["position"] == pos]

    X_test = test.drop(labels=["pathogenicity", "position"], axis=1, inplace=False)
    y_test = test["pathogenicity"]

    X_train = train.drop(labels=["pathogenicity", "position"], axis=1, inplace=False)
    y_train = train["pathogenicity"]
    
    # Oversample the train set using SMOTE.
    oversample = SMOTE(sampling_strategy='minority', random_state=42)
    X_train_resampled, y_train_resampled = oversample.fit_resample(X_train, y_train)
    # class_distribution = y_train_resampled.value_counts()
    # print(f"Training set: (SMOTE)\n{class_distribution}")

    # Initialize and fit XGBoost model.
    xgb_classifier = xgb.XGBClassifier(learning_rate=0.1, max_depth=3, n_estimators=100, random_state=42)
    xgb_classifier.fit(X_train_resampled, y_train_resampled)  # Fit the model with the resampled data
    
    # Predict probabilities (Instead of predicting binary values)
    prob = xgb_classifier.predict_proba(X_test)[:, 1]
    y_pred = xgb_classifier.predict(X_test)

    # Store probabilities, predictions, and ground truth labels
    probabilities.extend(prob)
    predictions.extend(y_pred)
    y_tests.extend(y_test)
    
    # Evaluate the model performance.
    tp = sum((y_test == 1) & (y_pred == 1))
    fp = sum((y_test == 0) & (y_pred == 1))
    fn = sum((y_test == 1) & (y_pred == 0))
    tn = sum((y_test == 0) & (y_pred == 0))

    print(f"TP: {tp}, FP: {fp}, TN: {tn}, FN: {fn}")

    tps.append(tp)
    fps.append(fp)
    fns.append(fn)
    tns.append(tn)

    print(f"tps: {sum(tps)}, fps: {sum(fps)}, fns: {sum(fns)}, tns: {sum(tns)}")

    print(f"Prediction: {y_pred}. Reality: {y_test.values}")

# Calculate MCC
TP = sum(tps)
FP = sum(fps)
FN = sum(fns)
TN = sum(tns)
mcc = (TP * TN - FP * FN) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))

print(f"TP: {sum(tps)}, FP: {sum(fps)}, TN: {sum(fns)}, FN: {sum(tns)}")

sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)
precision = TP / (TP + FP)
accuracy = (TP + TN) / (TP + TN + FP + FN)
print(f"Results for {gene}:")
print(f"Sensitivity (Recall): {sensitivity}")
print(f"Specificity: {specificity}")
print(f"Precision: {precision}")
print(f"MCC for {gene}: {mcc}")
print(f"Accuracy: {accuracy}")

Position: 1 (1/921)
TP: 0, FP: 0, TN: 5, FN: 0
tps: 0, fps: 0, fns: 0, tns: 5
Prediction: [0 0 0 0 0]. Reality: [0 0 0 0 0]
Position: 2 (2/921)
TP: 0, FP: 0, TN: 6, FN: 0
tps: 0, fps: 0, fns: 0, tns: 11
Prediction: [0 0 0 0 0 0]. Reality: [0 0 0 0 0 0]
Position: 3 (3/921)
TP: 0, FP: 0, TN: 6, FN: 0
tps: 0, fps: 0, fns: 0, tns: 17
Prediction: [0 0 0 0 0 0]. Reality: [0 0 0 0 0 0]
Position: 4 (4/921)
TP: 0, FP: 0, TN: 5, FN: 0
tps: 0, fps: 0, fns: 0, tns: 22
Prediction: [0 0 0 0 0]. Reality: [0 0 0 0 0]
Position: 5 (5/921)
TP: 0, FP: 0, TN: 7, FN: 0
tps: 0, fps: 0, fns: 0, tns: 29
Prediction: [0 0 0 0 0 0 0]. Reality: [0 0 0 0 0 0 0]
Position: 6 (6/921)
TP: 0, FP: 0, TN: 3, FN: 0
tps: 0, fps: 0, fns: 0, tns: 32
Prediction: [0 0 0]. Reality: [0 0 0]
Position: 9 (7/921)
TP: 0, FP: 0, TN: 1, FN: 0
tps: 0, fps: 0, fns: 0, tns: 33
Prediction: [0]. Reality: [0]
Position: 15 (8/921)
TP: 0, FP: 0, TN: 2, FN: 0
tps: 0, fps: 0, fns: 0, tns: 35
Prediction: [0 0]. Reality: [0 0]
Position: 19 (9/921)

In [ ]:
from sklearn.metrics import roc_curve, auc

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_tests, probabilities)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
####### Getting metrics for current protein and adding to metrics_gene_specific_predictors #######

# metrics = pd.DataFrame(columns=["gene", "sensitivity", "specificity", "precision", "accuracy", "mcc", "num_variants", "num_benign", "num_pathogenic"])
# metrics.to_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\predictions_vs_real\\gene_specific_predictors\\metrics.csv", index=False)

metrics = pd.read_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\predictions_vs_real\\gene_specific_predictors\\metrics_gene_specific_predictors.csv")

protein_metrics = pd.DataFrame({
    "gene": [protein],
    "sensitivity": [sensitivity],
    "specificity": [specificity],
    "precision": [precision],
    "accuracy": [accuracy],
    "mcc": [mcc]
})
protein_metrics["num_variants"] = len(data)
protein_metrics["num_benign"] = sum(data["pathogenicity"] == 0)
protein_metrics["num_pathogenic"] = sum(data["pathogenicity"] == 1)
# add TP, FP, FN, TN to the dataframe
protein_metrics["TP"] = TP
protein_metrics["FP"] = FP
protein_metrics["FN"] = FN
protein_metrics["TN"] = TN
metrics = pd.concat([metrics, protein_metrics], ignore_index=True)
# add the number of variants for each protein
metrics.to_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\predictions_vs_real\\gene_specific_predictors\\metrics_gene_specific_predictors.csv", index=False)


In [ ]:
from sklearn.metrics import precision_recall_curve

# create a graph showing the precision for each threshold
precision, recall, thresholds = precision_recall_curve(y_tests, probabilities)
plt.plot(thresholds, precision[:-1], label="Precision")
plt.plot(thresholds, recall[:-1], label="Recall")
plt.xlabel("Threshold")
plt.ylabel("Precision/Recall")
plt.legend()
plt.show()



In [15]:
thresholds

array([0.00274846, 0.00279623, 0.00390097, 0.00506073, 0.00603197,
       0.00617171, 0.00642017, 0.00680564, 0.00703071, 0.0073544 ,
       0.0075057 , 0.00782517, 0.00821149, 0.00827537, 0.00848366,
       0.00879773, 0.00914296, 0.01005938, 0.01021592, 0.01038297,
       0.01055659, 0.01081134, 0.0110833 , 0.01127141, 0.01146773,
       0.01185433, 0.01227873, 0.01286814, 0.01325669, 0.01380633,
       0.01411296, 0.01444434, 0.01454212, 0.01541893, 0.01566567,
       0.01618382, 0.01683782, 0.01717573, 0.01874467, 0.01930415,
       0.02012933, 0.02157421, 0.02306954, 0.02387608, 0.02424583,
       0.02588293, 0.02669196, 0.0271423 , 0.02770322, 0.02952178,
       0.03035793, 0.03084195, 0.03261774, 0.03319075, 0.0339596 ,
       0.03484694, 0.03618979, 0.03707642, 0.03741371, 0.04061849,
       0.04065758, 0.04291777, 0.04466041, 0.04548116, 0.04601038,
       0.04686188, 0.04885734, 0.04968765, 0.05230929, 0.05478719,
       0.05507173, 0.05614475, 0.0579485 , 0.05911886, 0.06339

In [19]:
len([prob for prob in probabilities if prob >= 0.994773])

9